In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [3]:
driver = webdriver.Chrome()

In [4]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[2]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 삼성 라이온즈 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[6]').click()

# 규정 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_reg"]/button').click()
# 규정 > 전체 선택
driver.find_element(By.XPATH, '//*[@id="select_reg"]/ul/li[5]').click()

In [8]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [9]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [10]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])
print(len(p))

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', '비율', '비율', '비율', '비율', '비율', '비율', 'WAR▼']
1 ['Rank', 'Name', 'Team', 'WAR', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'wRC+', 'WAR▼']
2 ['1', '김영웅', '24  SS', '1.89', '40', '173', '172', '153', '24', '46', '8', '1', '9', '83', '26', '3', '0', '18', '1', '0', '50', '2', '1', '0', '0.301', '0.378', '0.542', '0.920', '0.038', '125.8', '1.89']
3 ['2', '구자욱', '24  LF', '1.71', '39', '179', '178', '158', '26', '49', '10', '0', '8', '83', '33', '3', '2', '13', '6', '1', '26', '1', '0', '2', '0.310', '0.380', '0.525', '0.905', '0.046', '132.0', '1.71']
4 ['3', '맥키넌', '24  1B', '1.56', '36', '162', '160', '138', '17', '50', '7', '0', '3', '66', '19', '1', '0', '24', '0', '3', '15', '3', '0', '0', '0.362', '0.457', '0.478', '0.

In [11]:
num = (len(p) // 10)-1
start_num = 2
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [17]:
data=pd.DataFrame(p)
data = data[2:]
data.columns=["Rank","Name","Position","WAR","G","PA","ePA","AB","R","H","2B","3B","HR","TB","RBI","SB","CS","BB","HP","IB","SO","GDP","SH","SF","AVG","OBP","SLG","OPS","R/ePA","wRC+","WAR2"]
data['Position'].replace({"24  " : ""}, regex=True, inplace=True)
data['Team']="Samsung"
data.drop(columns=["WAR2"], inplace=True)
print(data)

   Rank Name Position    WAR   G   PA  ePA   AB   R   H  ... GDP SH SF    AVG  \
2     1  김영웅       SS   1.89  40  173  172  153  24  46  ...   2  1  0  0.301   
3     2  구자욱       LF   1.71  39  179  178  158  26  49  ...   1  0  2  0.310   
4     3  맥키넌       1B   1.56  36  162  160  138  17  50  ...   3  0  0  0.362   
5     4  김지찬       CF   1.30  41  169  165  143  24  40  ...   1  4  0  0.280   
6     5  이성규       CF   1.12  38   89   86   68  12  16  ...   1  3  2  0.235   
7     6  이재현       SS   1.10  24  108  108   94  15  26  ...   1  0  1  0.277   
8     7  김헌곤       RF   0.44  37   78   77   70  11  23  ...   1  1  0  0.329   
9     8  강민호        C   0.27  41  134  133  119  14  31  ...   8  0  1  0.261   
10    9  김재혁       LF   0.27  12   34   34   30   5   7  ...   0  0  2  0.233   
11   10  류지혁       2B   0.23  23   81   81   71   9  20  ...   0  0  1  0.282   
12   11  전병우       3B   0.20   4   12   12   10   1   4  ...   0  0  0  0.400   
13   12  류승민       LF   0.13

In [18]:
data.to_csv("Samsung_hit.csv", index=False)